In [2]:
%%writefile requirements.txt
numpy
pandas==2.2.3
matplotlib==3.10.0

Overwriting requirements.txt


In [3]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.5 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 37.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 15.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 32.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 17.7 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


First, read in the csv files.

In [5]:
# TODO: Read in credit train and credit test csv
df1 = pd.read_csv('archive/credit_train.csv')
df2 = pd.read_csv('archive/credit_test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'archive/credit_train.csv'

In [ ]:
# TODO: Return the first 5 rows of the train dataframe
df1.head()

In [ ]:
# TODO: Return a summary of the train dataframe
df1.info()

In [ ]:
#Drops duplicates and unnecessary columns
df1 = df1.drop_duplicates()
df1 = df1.drop(columns=['Loan ID', 'Customer ID'], axis=1)

Then, handle all null values.

In [ ]:
# TODO: Check for missing values in the train dataframe (Hint: use isnull().sum())
df1.isnull().sum()

In [ ]:
# standardize nulls to np.nan, so pandas recognizes them as missing
df1.replace(['nan','NaN','NULL','',' '], np.nan, inplace=True)

# TODO: Impute missing values (Hint: Use fillna())
# Hint: you can use median, mode, mean, or a specific value to fill in missing values
df1['Current Loan Amount'].fillna(0, inplace=True)
df1['Years of Credit History'].fillna(df1['Years of Credit History'].median(), inplace=True)

In [ ]:
# TODO: Check for missing values again to confirm they were filled
df1.isnull().sum()

Feature Engineering

In [ ]:
import re
import numpy as np
import pandas as pd


# 1a) Years in current job: extract numeric and bin
def extract_years(s):
    s = str(s)
    match = re.findall(r'\d+', s)
    if match:
        return int(match[0])
    return None

df1['Years in current job'] = df1['Years in current job'].apply(extract_years)

bins = [0, 2, 5, 10, float('inf')]
labels = ['0-2', '2-5', '5-10', '10+']
df1['Years in current job'] = pd.cut(df1['Years in current job'], bins=bins, labels=labels, right=False)

# 1b) Standardize Purpose values to the canonical list used in the reference
df1['Purpose'] = df1['Purpose'].replace({
    'Home Improvements':'Home Renovation / Improvement',
    'Debt Consolidation':'Debt Consolidation',
    'Buy House':'Home Purchase / Mortgage',
    'Business Loan':'Business / Startup Capital',
    'Buy a Car':'Car / Vehicle Purchase',
    'Take a Trip':'Vacation / Travel',
    'small_business':'Business / Startup Capital',
    'Medical Bills':'Medical Expenses',
    'wedding':'Wedding Expenses',
    'vacation':'Vacation / Travel',
    'Educational Expenses':'Education / Tuition Fees',
    'moving':'Moving / Relocation Costs',
    'other':'Other / Miscellaneous',
    'Other':'Other / Miscellaneous',
    'renewable_energy':'Buying Electronics / Appliances',
    'major_purchase':'Business / Startup Capital'
})

Then, encode the categorical data.

In [ ]:
#Separate categorical and numerical columns
cat_list = df1.select_dtypes(include='object').columns.tolist()
num_list = df1.select_dtypes(exclude='object').columns.tolist()
print("Categorical:", cat_list)
print("Numerical:", num_list)

In [1]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder

# 2a) Ordinal encoding for 'Years in current job'
od = OrdinalEncoder(categories=[['0-2', '2-5', '5-10', '10+']])
df1['Years in current job'] = od.fit_transform(df1[['Years in current job']])

# 2b) Label encode simple categories
le = LabelEncoder()
for col in ['Loan Status', 'Term', 'Home Ownership']:
    df1[col] = le.fit_transform(df1[col].fillna('Missing'))

# 2c) One-hot encode 'Purpose' 
# TODO: One-hot encode 'Purpose' (Hint: use OneHotEncoder from sklearn)
ohe = OneHotEncoder(sparse_output=False, drop='first')
purpose_encoded = ohe.fit_transform(df1[['Purpose']].fillna('Missing'))
purpose_df = pd.DataFrame(purpose_encoded, columns=ohe.get_feature_names_out(['Purpose']), index=df1.index)
df1 = pd.concat([df1.drop('Purpose', axis=1), purpose_df], axis=1)

ModuleNotFoundError: No module named 'sklearn'

Finally, create a train/test split.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TODO: Create feature matrix X and target vector y
X = df1.drop('Loan Status', axis=1)
y = df1['Loan Status']

# TODO: Create a train/test split (Hint: use train_test_split from sklearn)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify = y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Save the processed data to CSV files
import pandas as pd

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df  = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_scaled_df.to_csv("X_train_scaled.csv", index=False)
X_test_scaled_df.to_csv("X_test_scaled.csv", index=False)

y_train.to_csv("y_train.csv", index=False, header=["Loan Status"])
y_test.to_csv("y_test.csv", index=False, header=["Loan Status"])